### Importaciones

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, lower, explode, regexp_replace, size #Para limpiar las palabras
import re #Para los archivos
import math #Para matematicas complejas (logaritmos)
import nltk #Ejecutar solo una vez para instalar | Para las stopwords
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("stopwords")
from nltk.tokenize import word_tokenize #Para tokenizar
from nltk.corpus import stopwords #Para las Stopwords

[nltk_data] Downloading package punkt to /home/gael-
[nltk_data]     guzman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/gael-
[nltk_data]     guzman/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/gael-
[nltk_data]     guzman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Sesion de spark

In [2]:
spark = SparkSession.builder.appName("RecomendacionLibros").master("local[*]").getOrCreate()
sc = spark.sparkContext

25/12/10 23:29:28 WARN Utils: Your hostname, gael-guzman-B550MH-3-0 resolves to a loopback address: 127.0.1.1; using 192.168.1.3 instead (on interface enp3s0)
25/12/10 23:29:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 23:29:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/10 23:29:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/10 23:29:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Cargar libros

In [3]:
rdd = sc.wholeTextFiles("Libros_clean/*.txt")
rdd = rdd.map(lambda x: (re.findall(r"[^/]+$", x[0])[0], x[1]))

### Limpieza de palabras

In [4]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9 \n]', '', text) #Simbolos y caracteres especiales
    return text.lower()                         #Minusculas

rdd_clean = rdd.map(lambda x: (x[0], clean_text(x[1]))) #Formato: Titulo, Contenido

### Quitar stopwords

In [5]:
stopwords = set(stopwords.words("english")) #Idioma de los documentos
def quitar_stopwords(texto):
 if not texto:
  return []

 # Tokenizar
 tokens = word_tokenize(texto.lower())

 # Filtrar palabras alfabéticas y sin stopwords
 tokens = [t for t in tokens if t.isalpha()]
 tokens = [t for t in tokens if t not in stopwords]

 return tokens

In [6]:
rdd_tokens = rdd_clean.map(
    lambda x: (x[0], quitar_stopwords(x[1]))
)

### Agregar '1' a las palabras

In [7]:
#Formato ((Libro, palabra), 1)
rdd_cont = rdd_tokens.flatMap(lambda x: [((x[0], w), 1) for w in x[1]])
rdd_cont.take(3)

[(('A_Christmas_Carol_by_Charles_Dickens.txt', 'ebook'), 1),
 (('A_Christmas_Carol_by_Charles_Dickens.txt', 'christmas'), 1),
 (('A_Christmas_Carol_by_Charles_Dickens.txt', 'carol'), 1)]

### Agrupar palabras iguales

In [8]:
#Formato ((Libro, palabra), Total)
rdd_agrup = rdd_cont.reduceByKey(lambda a, b: a + b)
rdd_agrup.take(5)

25/12/10 23:29:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

[(('A_Christmas_Carol_by_Charles_Dickens.txt', 'christmas'), 89),
 (('A_Christmas_Carol_by_Charles_Dickens.txt', 'carol'), 5),
 (('A_Christmas_Carol_by_Charles_Dickens.txt', 'suzanne'), 1),
 (('A_Christmas_Carol_by_Charles_Dickens.txt', 'shell'), 1),
 (('A_Christmas_Carol_by_Charles_Dickens.txt', 'janet'), 1)]

### Total de palabras en el libro

In [9]:
#Formato (Libro, TotalPalabras)
rdd_total = rdd_agrup.map(lambda x: (x[0][0], x[1])).reduceByKey(lambda a, b: a + b)
rdd_total.take(3)

[('A_Christmas_Carol_by_Charles_Dickens.txt', 14311),
 ('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
  13847),
 ('A_Study_in_Scarlet_by_Arthur_Conan_Doyle.txt', 19950)]

### Calcular DF de las palabras

In [10]:
rdd_para_df = rdd_tokens.flatMap(lambda x: [(w, x[0]) for w in set(x[1])])
rdd_df = rdd_para_df.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)
rdd_df.take(3)

[('rapid', 66), ('flung', 58), ('lessons', 53)]

### Contar documentos

In [11]:
TotalDocumentos = rdd_tokens.count()
print(TotalDocumentos)

[Stage 8:=============================>                             (1 + 1) / 2]

98


### TF Normalizado

In [12]:
rdd_normalizado = rdd_agrup.map(lambda x: (x[0][0], (x[0][1], x[1]))).join(rdd_total)\
                           .map(lambda x: ((x[0], x[1][0][0]), x[1][0][1] / x[1][1]))
rdd_normalizado.take(3)

[(('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'christmas'),
  0.006355167184227631),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'carol'),
  0.0002888712356467105),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'story'),
  0.0002888712356467105)]

### Calcular pesos

In [13]:
rdd_df_map = rdd_df.collectAsMap()
rdd_tfidf = rdd_normalizado.map(lambda x: ((x[0][0], x[0][1]), x[1] * 
                                           math.log(TotalDocumentos / rdd_df_map[x[0][1]])))
rdd_tfidf.take(3)

[(('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'christmas'),
  0.004150825626003442),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'carol'),
  0.0005421874131778325),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'story'),
  4.111106295378216e-05)]

### Vectorizar

In [14]:
rdd_vectores = rdd_tfidf.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey()\
                         .mapValues(lambda vals: dict(vals))
#Muestra el valor para cada palabra dentro del libro. !!!NO MOSTRAR¡¡¡ #rdd_vectores.take(1)

### Calcular similitud por coseno

In [15]:
#Funcion para calcular la similitud
def coseno(v1, v2):
    palabras = set(v1.keys()).union(v2.keys())
    dot = sum(v1.get(p, 0) * v2.get(p, 0) for p in palabras)
    norm1 = math.sqrt(sum(v1.get(p, 0)**2 for p in palabras))
    norm2 = math.sqrt(sum(v2.get(p, 0)**2 for p in palabras))
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot / (norm1 * norm2)

In [16]:
#Realizar comparaciones entre libros
rdd_pairs = rdd_vectores.cartesian(rdd_vectores)\
                        .filter(lambda x: x[0][0] < x[1][0])  # Evita pares duplicados (Libro1-Libro1)

rdd_simil = rdd_pairs.map(lambda x: ((x[0][0], x[1][0]), coseno(x[0][1], x[1][1])))
rdd_simil.take(3)

[(('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   "Alice's_Adventures_in_Wonderland_by_Lewis_Carroll.txt"),
  0.002860474230362577),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   'Anna_Karenina_by_graf_Leo_Tolstoy.txt'),
  0.002925165314445679),
 (('A_Christmas_Carol_in_Prose;_Being_a_Ghost_Story_of_Christmas_by_Charles_Dickens.txt',
   "Dio's_Rome,_Volume_1_by_Cassius_Dio_Cocceianus.txt"),
  0.0013142938221323657)]

### Matriz de similitud

In [17]:
# rdd_simil = RDD con: ((doc1, doc2), sim)

rdd_simetrico = rdd_simil.flatMap(
    lambda x: [
        (x[0][0], (x[0][1], x[1])),   # Doc1 → (Doc2, sim)
        (x[0][1], (x[0][0], x[1]))    # Doc2 → (Doc1, sim)
    ]
)

# Agrupa los valores por documento
rdd_grupos = rdd_simetrico.groupByKey()

# Convierte a diccionario Python: {doc: [(doc2, sim), ...]}
sim_dict = rdd_grupos.mapValues(list).collectAsMap()
#Contiene un libro y su parecido con todos los otros No imprimir de preferencia list(sim_dict.items())[:1]

### Funciones

In [18]:
# Función: libros más parecidos
def libros_parecidos():
    doc = input("Ingrese el nombre del documento: ")
    n = int(input("Ingrese la cantidad de libros parecidos que desea: "))
    
    if doc not in sim_dict:
        print("Documento no encontrado.")
        return
    
    similitudes_doc = sim_dict[doc]  # Lista de pares (otro_doc, similitud)
    
    # Ordenar por similitud descendente
    top_docs = sorted(similitudes_doc, key=lambda x: x[1], reverse=True)[:n]
    
    print(f"\nLos {n} libros más parecidos a '{doc}' son:")
    for otro_doc, sim in top_docs:
        print(f"{otro_doc}: {sim:.4f}")

# Función: cantidad de palabras que describen un documento
vec_dict = rdd_vectores.collectAsMap()

def cantidad_palabras():
    doc = input("Ingrese el nombre del documento: ")
    n = int(input("Ingrese la cantidad de palabras que desea ver que describen el documento: "))
    
    if doc not in vec_dict:
        print("Documento no encontrado.")
        return
    
    vec = vec_dict[doc]  # Diccionario de palabras y TF-IDF
    
    # Ordenar las palabras por TF-IDF descendente y tomar las top n
    top_words = sorted(vec.items(), key=lambda x: x[1], reverse=True)[:n]
    
    print(f"\nLas {n} palabras que describen mejor el documento '{doc}' son:")
    for palabra, tfidf in top_words:
        print(f"{palabra} : {tfidf:.4f}")

In [20]:
libros_parecidos()
cantidad_palabras()

Ingrese el nombre del documento:  A_Tale_of_Two_Cities_by_Charles_Dickens.txt
Ingrese la cantidad de libros parecidos que desea:  5



Los 5 libros más parecidos a 'A_Tale_of_Two_Cities_by_Charles_Dickens.txt' son:
The_Adventures_of_Roderick_Random_by_T._Smollett.txt: 0.0310
The_Works_of_Edgar_Allan_Poe_—_Volume_2_by_Edgar_Allan_Poe.txt: 0.0259
Jane_Eyre_An_Autobiography_by_Charlotte_Brontë.txt: 0.0239
The_Interesting_Narrative_of_the_Life_of_Olaudah_Equiano,_Or_Gustavus_Vassa,_The_African_by_Equiano.txt: 0.0236
Romantic_castles_and_palaces_.txt: 0.0222


Ingrese el nombre del documento:  A_Tale_of_Two_Cities_by_Charles_Dickens.txt
Ingrese la cantidad de palabras que desea ver que describen el documento:  10



Las 10 palabras que describen mejor el documento 'A_Tale_of_Two_Cities_by_Charles_Dickens.txt' son:
lorry : 0.0205
defarge : 0.0201
manette : 0.0111
pross : 0.0107
darnay : 0.0100
carton : 0.0090
lucie : 0.0089
cruncher : 0.0077
stryver : 0.0074
tellsons : 0.0060
